<a href="https://colab.research.google.com/github/michael-L-i/CS229-Final-Project/blob/main/EXIF_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")



Mounted at /content/drive


In [ ]:
import pandas as pd
import sklearn
import torch


In [ ]:
real_exif = pd.read_csv("/content/drive/My Drive/School/Year 2024–2025/CS229/final_project/EXIF Dataset/real_exif.csv")
fake_exif = pd.read_csv("/content/drive/My Drive/School/Year 2024–2025/CS229/final_project/EXIF Dataset/fake_exif.csv")

In [ ]:
real_exif = real_exif[["AEAverage", "Aperture", "BrightnessValue", "ColorTemperature", "ExposureTime", "FNumber", "FocalLength", "ISO", "LightValue", "ShutterSpeed"]]
fake_exif = fake_exif[["AEAverage", "Aperture", "BrightnessValue", "ColorTemperature", "ExposureTime", "FNumber", "FocalLength", "ISO", "LightValue", "ShutterSpeed"]]

real_exif["Label"] = 0
fake_exif["Label"] = 1

real_exif


<ipython-input-38-fd000ee57820>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_exif["Label"] = 0
<ipython-input-38-fd000ee57820>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_exif["Label"] = 1


,AEAverage,Aperture,BrightnessValue,ColorTemperature,ExposureTime,FNumber,FocalLength,ISO,LightValue,ShutterSpeed,Label
0,146.0,1.8,-0.893535,3333.0,1/40,1.8,6.8 mm,800,4.0,1/40,0
1,194.0,1.8,10.649945,5317.0,1/7937,1.8,6.8 mm,80,14.9,1/7937,0
2,200.0,1.8,2.709466,3293.0,1/60,1.8,6.8 mm,160,6.9,1/60,0
3,252.0,1.8,1.238959,3390.0,1/60,1.8,6.8 mm,500,5.2,1/60,0
4,193.0,1.8,8.437375,5201.0,1/1721,1.8,6.8 mm,80,12.7,1/1721,0
...,...,...,...,...,...,...,...,...,...,...,...
83,78.0,1.8,-3.251055,5393.0,1/25,1.8,6.8 mm,1600,2.3,1/25,0
84,180.0,1.8,11.634730,5317.0,1/20000,1.8,6.8 mm,100,16.0,1/20000,0
85,179.0,1.8,2.636222,5849.0,1/93,1.8,6.8 mm,200,7.2,1/93,0
86,166.0,1.8,8.891683,5372.0,1/2710,1.8,6.8 mm,80,13.4,1/2710,0


In [ ]:
# convert everything to floats

real_exif["FocalLength"] = real_exif["FocalLength"].str.replace(" mm", "", regex=False).astype(float)
real_exif["ExposureTime"] = real_exif["ExposureTime"].apply(eval)
real_exif["ShutterSpeed"] = real_exif["ShutterSpeed"].apply(eval)

fake_exif["FocalLength"] = fake_exif["FocalLength"].str.replace(" mm", "", regex=False).astype(float)
fake_exif["ExposureTime"] = fake_exif["ExposureTime"].apply(eval)
fake_exif["ShutterSpeed"] = fake_exif["ShutterSpeed"].apply(eval)



In [ ]:
df = pd.concat([real_exif, fake_exif], ignore_index=True)

In [ ]:
from sklearn.model_selection import train_test_split

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,shuffle=True)

y_train.head()

,Label
30,0
65,0
110,1
47,0
169,1


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

clf = RandomForestClassifier(max_depth=2)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision}")
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall}")

Accuracy: 0.8918918918918919
Precision: 0.8260869565217391
Recall: 1.0


In [ ]:
# Get Feature Importances
importances = clf.feature_importances_

# Convert to DataFrame for better visualization
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print(feature_importance_df)

            Feature  Importance
3  ColorTemperature    0.262973
4      ExposureTime    0.204023
9      ShutterSpeed    0.193133
2   BrightnessValue    0.171882
8        LightValue    0.099647
7               ISO    0.048282
0         AEAverage    0.012502
1          Aperture    0.004414
5           FNumber    0.001656
6       FocalLength    0.001489
